# Chain Prompting with Rotten Tomatoes Reviews

In [4]:
#!pip install gradio

  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5595 sha256=89342df15e3caf33f05a853459f8b8d6d1bce2cff779c827fee5a5a0253d9695
  Stored in directory: c:\users\allen\appdata\local\pip\cache\wheels\1f\f1\8d\367922b023b526b7c2ced5db30932def7b18cf39d7ac6e8572
Successfully built ffmpy
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [3]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [4]:
df = pd.read_csv('rotten_tomatoes_critic_reviews.csv')

In [5]:
df.head()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [6]:
llm = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo")

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in less than 10 words:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="Summary"
                    )


In [7]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you translate the following sentence into French:"
    "\n\n{Summary}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="Translation"
                    )

In [8]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two],
    input_variables=["Review"],
    output_variables=["Summary","Translation"],
    verbose=True
)

In [9]:
review = df.review_content[3]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Whether audiences will get behind The Lightning Thief is hard to predict. Overall, it's an entertaining introduction to a promising new world -- but will the consuming shadow of Potter be too big to break free of?",
 'Summary': 'Entertaining introduction to a promising new world, overshadowed by Potter.',
 'Translation': 'Introduction divertissante à un nouveau monde prometteur, éclipsée par Potter.'}

## Deployment on Gradio.app

In [10]:
import gradio as gr


def llm(input):
    return overall_chain(input)

demo = gr.Interface(fn=llm, inputs="text", outputs="text")
    
demo.launch(share = True)  

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://b1b5c02224e489bd72.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new SequentialChain chain...

> Finished chain.


> Entering new SequentialChain chain...

> Finished chain.
